In [60]:
import pandas as pd
from io import StringIO
import re

In [61]:
# source folder
kerg2_2021_csv = r'..\resources\electionData\sourceCSV\2021\kerg2.csv'
kerg2_2017_csv = r'..\resources\electionData\sourceCSV\2017\kerg2.csv'

bevoelkerungszahlen_2017_csv = r'..\resources\electionData\sourceCSV\2017\bevoelkerungszahlen_2017.csv'
bevoelkerungszahlen_2021_csv = r'..\resources\electionData\sourceCSV\2021\bevoelkerungszahlen_2021.csv'

parteien_2021_mitLandesliste_txt = r'..\resources\electionData\sourceCSV\2021\parteien_2021_mitLandesliste.txt'
parteien_2021_ohneLandesliste_txt = r'..\resources\electionData\sourceCSV\2021\parteien_2021_ohneLandesliste.txt'
parteien_2017_mitLandesliste_txt = r'..\resources\electionData\sourceCSV\2017\parteien_2017_mitLandesliste.txt'
parteien_2017_ohneLandesliste_txt = r'..\resources\electionData\sourceCSV\2017\parteien_2017_ohneLandesliste.txt'

kandidaturen_2021_csv = r'..\resources\electionData\sourceCSV\2021\kandidaturen_2021.csv'
kandidaturen_2017_csv = r'..\resources\electionData\sourceCSV\2017\kandidaten_2017.csv'

# target folder

bundesLand_csv = r'..\resources\electionData\targetCSV\bundesland.csv'
bevoelkerung_csv = r'..\resources\electionData\targetCSV\bevoelkerung.csv'

wahlkreise_csv = r'..\resources\electionData\targetCSV\wahlkreise.csv'

parteien_csv = r'..\resources\electionData\targetCSV\parteien.csv'
parteienTeilnahme_csv = r'..\resources\electionData\targetCSV\parteienTeilnahme.csv'


kandidaten_csv = r'..\resources\electionData\targetCSV\kandidaturen.csv'
direktmandate_csv = r'..\resources\electionData\targetCSV\direktmandate.csv'
landesliste_csv = r'..\resources\electionData\targetCSV\landesliste.csv'

erststimme_aggr_csv = r'..\resources\electionData\targetCSV\erststimmeAggr.csv'
zweitstimme_aggr_csv = r'..\resources\electionData\targetCSV\zweitstimmeAggr.csv'

stimmen_csv = r'..\resources\electionData\targetCSV\stimmen.csv'




In [62]:
# Bundesländer

kerg2 = pd.read_csv(kerg2_2021_csv, sep=';', skiprows=9)

kerg2 = kerg2.loc[kerg2['Gebietsart'] == 'Land'] # filter for bundesland only
kerg2 = kerg2.filter(items=['Gebietsnummer', 'Gebietsname']) # drop unecessary columns
kerg2 = kerg2.drop_duplicates(subset='Gebietsnummer') 
kerg2 = kerg2.sort_values(by = 'Gebietsnummer')
kerg2 = kerg2.rename(columns={'Gebietsnummer':'id', 'Gebietsname': 'name'})

kerg2.to_csv(bundesLand_csv, index=False, sep=';')

In [63]:
# Bundesländer bevoelkerung

bundesLänder = pd.read_csv(bundesLand_csv, sep=';')
bevoelkerung = pd.read_csv(bevoelkerungszahlen_2021_csv, sep=';')

merged_2021 = bundesLänder.merge(right=bevoelkerung, on='name')
merged_2021 = merged_2021.drop('name', axis=1)
merged_2021['jahr'] = 2021

bundesLänder = pd.read_csv(bundesLand_csv, sep=';')
bevoelkerung = pd.read_csv(bevoelkerungszahlen_2017_csv, sep=';')

merged_2017 = bundesLänder.merge(right=bevoelkerung, on='name')
merged_2017 = merged_2017.drop('name', axis=1)
merged_2017['jahr'] = 2017

merged = pd.concat([merged_2017, merged_2021])
merged = merged.rename(columns={'id': 'bundesland_id'})

merged.to_csv(bevoelkerung_csv, index=False, sep=';')

In [64]:
# parteien

def extractParteien(html: str) -> list[list[str]]:
    html = html.replace('\n', '')
    html = html.replace('\t', '')
    html = html.replace('</tr>', '')
    html = html.replace('</td>', '')
    html = html.replace('<br>', ' ')
    html = html.replace('&nbsp;', '')
    html = html.split('<tr>')[1:]
    html = [partei.split('<td>')[2:] for partei in html]
    return html


# 2017 https://www.bundeswahlleiterin.de/bundestagswahlen/2017/wahlbewerber.html
with open(parteien_2017_mitLandesliste_txt, mode ='r') as file:
    html = ''.join(file.readlines())
    parteien = extractParteien(html)
    parteien_2017_list = pd.DataFrame(parteien, columns=['kurzbezeichnung', 'name', 'Zusatzbezeichnung'])
    
with open(parteien_2017_ohneLandesliste_txt, mode ='r') as file:
    html = ''.join(file.readlines())
    parteien = extractParteien(html)
    parteien_2017_ohneList = pd.DataFrame(parteien, columns=['kurzbezeichnung', 'name', 'Zusatzbezeichnung'])

# hard coding missing name
parteien_2017_list.loc[parteien_2017_list['name'] == 'Allianz Deutscher Demokraten', 'kurzbezeichnung'] = 'ADD' 
parteien_2017_ohneList.loc[parteien_2017_ohneList['name'] == 'Neue Liberale – Die Sozialliberalen', 'kurzbezeichnung'] = 'NL' 

einzelBewerber_2017 = pd.read_csv(kandidaturen_2017_csv, sep= ';')
einzelBewerber_2017 = einzelBewerber_2017.dropna(axis=0, subset='Name, Vorname(n)')
einzelBewerber_2017['Gebietsnummer'] = einzelBewerber_2017['kandidiert im'].map(lambda s: int(next(iter(re.findall('Wahlkreis ([\\d]*)', s)), -1)))
einzelBewerber_2017['Partei'] = einzelBewerber_2017.apply(lambda row: row.Partei + ' '+ str(row.Gebietsnummer) if row.Partei.startswith('EB: ') else row.Partei, axis=1)
einzelBewerber_2017 = einzelBewerber_2017.loc[einzelBewerber_2017['Partei'].map(lambda s: s.startswith('EB: '))]
einzelBewerber_2017 = einzelBewerber_2017.filter(items=['Partei'])
einzelBewerber_2017 = einzelBewerber_2017.rename(columns={'Partei':'kurzbezeichnung'})
einzelBewerber_2017['name'] = ''
einzelBewerber_2017['Zusatzbezeichnung'] = ''

einzelBewerber_2017['isEinzelbewerber'] = True
parteien_2017_list['isEinzelbewerber'] = False
parteien_2017_ohneList['isEinzelbewerber'] = False

parteien_2017 = pd.concat([parteien_2017_list, parteien_2017_ohneList, einzelBewerber_2017], ignore_index=True)
parteien_2017['jahr'] = 2017


# 2021 https://www.bundeswahlleiterin.de/bundestagswahlen/2021/wahlbewerber.html
with open(parteien_2021_mitLandesliste_txt, mode ='r') as file:
    html = ''.join(file.readlines())
    parteien = extractParteien(html)
    parteien_2021_list = pd.DataFrame(parteien, columns=['kurzbezeichnung', 'name', 'Zusatzbezeichnung'])
    
with open(parteien_2021_ohneLandesliste_txt, mode ='r') as file:
    html = ''.join(file.readlines())
    parteien = extractParteien(html)
    parteien_2021_ohneList = pd.DataFrame(parteien, columns=['kurzbezeichnung', 'name', 'Zusatzbezeichnung'])

einzelBewerber_2021 = pd.read_csv(kandidaturen_2021_csv, sep= ';')
einzelBewerber_2021['Gruppenname'] = einzelBewerber_2021.apply(lambda row: row.Gruppenname + ' '+ str(row.Gebietsnummer) if row.Gruppenname.startswith('EB: ') else row.Gruppenname, axis=1)
einzelBewerber_2021 = einzelBewerber_2021.loc[einzelBewerber_2021['Gruppenname'].map(lambda s: s.startswith('EB: '))]
einzelBewerber_2021 = einzelBewerber_2021.filter(items=['Gruppenname', 'GruppennameLang'])
einzelBewerber_2021 = einzelBewerber_2021.rename(columns={'Gruppenname':'kurzbezeichnung','GruppennameLang': 'name'})
einzelBewerber_2021['Zusatzbezeichnung'] = ''

einzelBewerber_2021['isEinzelbewerber'] = True
parteien_2021_list['isEinzelbewerber'] = False
parteien_2021_ohneList['isEinzelbewerber'] = False

parteien_2021 = pd.concat([parteien_2021_list, parteien_2021_ohneList, einzelBewerber_2021], ignore_index=True)
parteien_2021.loc[parteien_2021['Zusatzbezeichnung'] == '–', 'Zusatzbezeichnung'] = ''
parteien_2021['jahr'] = 2021

# Zum vergleich von Namen - Ergebins: kurzbezeichnung ist einzigartig und fd: kurzbezeichnung -> name, Zusatzbezeichnung
#insgesamt = pd.concat([parteien_2021, parteien_2017], ignore_index=True)
#insgesamt = insgesamt.sort_values(by = 'kurzbezeichnung', ignore_index=True)
#insgesamt.to_csv("vergleichParteiNamen.csv")

insgesamt = pd.concat([parteien_2021, parteien_2017], ignore_index=True)
insgesamt = insgesamt.drop('jahr', axis=1)
insgesamt = insgesamt.drop_duplicates('kurzbezeichnung', ignore_index=True)
insgesamt.insert(loc=0, column='index', value=-1)
insgesamt['index'] = insgesamt.index.map(lambda i: i+1)

teilnahme_2017 = parteien_2017.filter(items=['kurzbezeichnung'])
teilnahme_2017 = teilnahme_2017.merge(right=insgesamt, on='kurzbezeichnung', how='inner')
teilnahme_2017 = teilnahme_2017.filter(items=['index'])
teilnahme_2017['jahr'] = 2017

teilnahme_2021 = parteien_2021.filter(items=['kurzbezeichnung'])
teilnahme_2021 = teilnahme_2021.merge(right=insgesamt, on='kurzbezeichnung', how='inner')
teilnahme_2021 = teilnahme_2021.filter(items=['index'])
teilnahme_2021['jahr'] = 2021

insgesamt.to_csv(parteien_csv, index=False, sep=';')

merged = pd.concat([teilnahme_2017, teilnahme_2021])
merged = merged.rename(columns={'index': 'partei_id'})
merged.to_csv(parteienTeilnahme_csv, index=False, sep=';')


In [65]:
# wahlkreise

kerg2 = pd.read_csv(kerg2_2021_csv, sep=';', skiprows=9)

kerg2 = kerg2.loc[kerg2['Gebietsart'] == 'Wahlkreis'] # filter for wahlkreis only
kerg2 = kerg2.filter(items=['Gebietsnummer', 'Gebietsname', 'UegGebietsnummer']) # drop unecessary columns
kerg2['UegGebietsnummer'] = kerg2['UegGebietsnummer'].astype('int')
kerg2 = kerg2.drop_duplicates(subset='Gebietsnummer') 
kerg2 = kerg2.sort_values(by = 'Gebietsnummer')
kerg2 = kerg2.rename(columns={'Gebietsnummer':'id', 'Gebietsname': 'name', 'UegGebietsnummer': 'bundesland_id'})

kerg2.to_csv(wahlkreise_csv, index=False, sep=';')

In [66]:
# kandidaturen

#Vornamen', 'Nachname', 'Geburtsjahr' um eindeutig Personen zu identifizieren

kandidaturen_2021 = pd.read_csv(kandidaturen_2021_csv, sep= ';')
kandidaturen_2021['Gruppenname'] = kandidaturen_2021.apply(lambda row: row.Gruppenname + ' '+ str(row.Gebietsnummer) if row.Gruppenname.startswith('EB: ') else row.Gruppenname, axis=1)
kandidaturen_2021 = kandidaturen_2021.filter(items=['Vornamen', 'Nachname', 'Geburtsjahr', 'Gruppenname'])
kandidaturen_2021 = kandidaturen_2021.drop_duplicates(subset=['Vornamen', 'Nachname', 'Geburtsjahr'], ignore_index=True)
kandidaturen_2021.insert(loc=0, column='id', value=-1)
kandidaturen_2021 = kandidaturen_2021.rename(columns={'Vornamen': 'vorname', 'Nachname': 'nachname', 'Geburtsjahr': 'geburtsjahr', 'Gruppenname': 'kurzbezeichnung'})
kandidaturen_2021['jahr'] = 2021

kandidaturen_2017 = pd.read_csv(kandidaturen_2017_csv, sep= ';')
kandidaturen_2017 = kandidaturen_2017.dropna(axis=0, subset='Name, Vorname(n)')
kandidaturen_2017['vorname'] = kandidaturen_2017['Name, Vorname(n)'].map(lambda s: s.split(', ')[1])
kandidaturen_2017['nachname'] = kandidaturen_2017['Name, Vorname(n)'].map(lambda s: s.split(', ')[0])
kandidaturen_2017['Gebietsnummer'] = kandidaturen_2017['kandidiert im'].map(lambda s: int(next(iter(re.findall('Wahlkreis ([\\d]*)', s)), -1)))
kandidaturen_2017['Partei'] = kandidaturen_2017.apply(lambda row: row.Partei + ' '+ str(row.Gebietsnummer) if row.Partei.startswith('EB: ') else row.Partei, axis=1)
kandidaturen_2017 = kandidaturen_2017.filter(items=['vorname', 'nachname', 'Geburts_jahr', 'Partei'])
kandidaturen_2017['Geburts_jahr'] = kandidaturen_2017['Geburts_jahr'].astype('int')
kandidaturen_2017 = kandidaturen_2017.rename(columns={'Geburts_jahr': 'geburtsjahr', 'Partei': 'kurzbezeichnung'})
kandidaturen_2017['jahr'] = 2017

kandidaturen_gesamt = pd.concat([kandidaturen_2021, kandidaturen_2017], ignore_index=True)

parteien = pd.read_csv(parteien_csv, sep=';')
parteien = parteien.rename(columns={'index':'partei_id'})
kandidaturen_gesamt = kandidaturen_gesamt.merge(right=parteien, on=['kurzbezeichnung'], how='inner')
kandidaturen_gesamt = kandidaturen_gesamt.filter(items=['id', 'vorname', 'nachname', 'geburtsjahr', 'partei_id', 'jahr'])

kandidaturen_gesamt['id']= kandidaturen_gesamt.index.map(lambda x: x + 1)
kandidaturen_gesamt.to_csv(kandidaten_csv, index=False, sep=';')

In [67]:
# direktmandate und listenplätze

# direktmandate 2021

kandidaturen = pd.read_csv(kandidaten_csv, sep= ';')
direktmandate_2021 = pd.read_csv(kandidaturen_2021_csv, sep= ';')

direktmandate_2021 = direktmandate_2021.loc[direktmandate_2021['Gebietsart'] == 'Wahlkreis']
direktmandate_2021['jahr'] = 2021
direktmandate_2021 = direktmandate_2021.rename(columns={'Vornamen': 'vorname', 'Nachname': 'nachname', 'Geburtsjahr': 'geburtsjahr'})
direktmandate_2021 = direktmandate_2021.merge(right=kandidaturen, on=['vorname', 'nachname', 'geburtsjahr', 'jahr'], how='inner')
direktmandate_2021 = direktmandate_2021.filter(items=['id', 'Gebietsnummer'])
direktmandate_2021 = direktmandate_2021.rename(columns={'id': 'kandidatur_id', 'Gebietsnummer': 'wahlkreis_id'})


# landeslisten 2021

kandidaturen = pd.read_csv(kandidaten_csv, sep= ';')
landesliste_2021 = pd.read_csv(kandidaturen_2021_csv, sep= ';')

landesliste_2021 = landesliste_2021.loc[landesliste_2021['Gebietsart'] == 'Land']
landesliste_2021['jahr'] = 2021
landesliste_2021 = landesliste_2021.rename(columns={'Vornamen': 'vorname', 'Nachname': 'nachname', 'Geburtsjahr': 'geburtsjahr'})
landesliste_2021 = landesliste_2021.merge(right=kandidaturen, on=['vorname', 'nachname', 'geburtsjahr', 'jahr'], how='inner')
landesliste_2021 = landesliste_2021.filter(items=['id', 'Gebietsnummer','' ,'Listenplatz'])
landesliste_2021 = landesliste_2021.rename(columns={'id': 'kandidatur_id', 'Gebietsnummer': 'bundesland_id', 'Listenplatz': 'listenplatz'})
landesliste_2021['listenplatz'] = landesliste_2021['listenplatz'].astype('int')

# direktmandate und listenplätze 2017
kandidaturen = pd.read_csv(kandidaten_csv, sep= ';')

with open(kandidaturen_2017_csv) as file:
    csv = ''.join(file.readlines())
    csv = csv.replace('\n;;;', '')

kandidaturen_2017 = pd.read_csv(StringIO(csv), sep= ';')

kandidaturen_2017['kandidiert im'].map(lambda s: s)
kandidaturen_2017['vorname'] = kandidaturen_2017['Name, Vorname(n)'].map(lambda s: s.split(', ')[1])
kandidaturen_2017['nachname'] = kandidaturen_2017['Name, Vorname(n)'].map(lambda s: s.split(', ')[0])
kandidaturen_2017['wahlkreis_id'] = kandidaturen_2017['kandidiert im'].map(lambda s: int(next(iter(re.findall('Wahlkreis ([\\d]*)', s)), -1)))
kandidaturen_2017['bundesland'] = kandidaturen_2017['kandidiert im'].map(lambda s: next(iter(re.findall('Land (.*?) \\(Platz', s)), -1))
kandidaturen_2017['listenplatz'] = kandidaturen_2017['kandidiert im'].map(lambda s: int(next(iter(re.findall('Land.*?\\(Platz\\xa0([\\d]*)', s)), -1)))
kandidaturen_2017['jahr'] = 2021
kandidaturen_2017 = kandidaturen_2017.rename(columns={'Geburts_jahr': 'geburtsjahr'})
kandidaturen_2017 = kandidaturen_2017.merge(right=kandidaturen, on=['vorname', 'nachname', 'geburtsjahr', 'jahr'], how='inner')

# direktmandate
direktmandate_2017 = kandidaturen_2017.filter(items=['id', 'wahlkreis_id'])
direktmandate_2017 = direktmandate_2017.loc[direktmandate_2017['wahlkreis_id'] > 0]
direktmandate_2017 = direktmandate_2017.rename(columns={'id': 'kandidatur_id', 'Gebietsnummer': 'wahlkreis_id'})

# listenplätze
bundesländer = pd.read_csv(bundesLand_csv, sep=';')
bundesländer = bundesländer.rename(columns={'name':'bundesland', 'id':'bundesland_id'})

landesliste_2017 = kandidaturen_2017.filter(items=['id', 'bundesland', 'listenplatz'])
landesliste_2017 = landesliste_2017.merge(right=bundesländer, on='bundesland', how='inner')
landesliste_2017 = landesliste_2017.filter(items=['id', 'bundesland_id', 'listenplatz']).rename(columns={'id':'kandidatur_id'})

direktmandate = pd.concat([direktmandate_2017, direktmandate_2021])
landesliste = pd.concat([landesliste_2017, landesliste_2021])

direktmandate.to_csv(direktmandate_csv, index=False, sep=';')
landesliste.to_csv(landesliste_csv, index=False, sep=';')


In [68]:
# zweitstimme aggr

kerg2_2021 = pd.read_csv(kerg2_2021_csv, sep=';', skiprows=9)
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Gebietsart'] == 'Wahlkreis']
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Stimme'] == 2]
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Gruppenart'] == 'Partei']
kerg2_2021 = kerg2_2021.filter(items=['Gruppenname', 'Gebietsnummer', 'Anzahl'])
kerg2_2021.insert(loc=2, column='jahr', value=0)
kerg2_2021['jahr'] = 2021
kerg2_2021['Anzahl'] = kerg2_2021['Anzahl'].map(lambda x: int(x) if not pd.isna(x) else 0)

kerg2_2017 = pd.read_csv(kerg2_2017_csv, sep=';', skiprows=9)
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Gebietsart'] == 'Wahlkreis']
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Stimme'] == 2]
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Gruppenart'] == 'Partei']
kerg2_2017 = kerg2_2017.filter(items=['Gruppenname', 'Gebietsnummer', 'Anzahl'])
kerg2_2017.insert(loc=2, column='jahr', value=0)
kerg2_2017['jahr'] = 2017
kerg2_2017['Anzahl'] = kerg2_2017['Anzahl'].map(lambda x: int(x) if not pd.isna(x) else 0)

merged = pd.concat([kerg2_2021,kerg2_2017], ignore_index=True)
merged = merged.rename(columns={'Gruppenname':'kurzbezeichnung', 'Gebietsnummer': 'wahlkreis_id', 'Anzahl': 'anzahl'})

parteien = pd.read_csv(parteien_csv, sep=';')
parteien = parteien.rename(columns={'index':'partei_id'})
merged = merged.merge(right=parteien, on=['kurzbezeichnung'], how='inner')
merged = merged.filter(items=['partei_id', 'wahlkreis_id','jahr', 'anzahl'])

merged.to_csv(zweitstimme_aggr_csv, sep=';', index=False)


In [69]:
# erstimme aggr
kerg2_2021 = pd.read_csv(kerg2_2021_csv, sep=';', skiprows=9)
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Gebietsart'] == 'Wahlkreis']
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Stimme'] == 1]
kerg2_2021 = kerg2_2021.loc[kerg2_2021['Gruppenart'] != 'System-Gruppe']
kerg2_2021['Gruppenname'] = kerg2_2021.apply(lambda row: row.Gruppenname + ' '+ str(row.Gebietsnummer) if row.Gruppenart == 'Einzelbewerber/Wählergruppe' else row.Gruppenname, axis=1)
kerg2_2021 = kerg2_2021.filter(items=['Gebietsnummer', 'Gruppenname', 'Anzahl'])
kerg2_2021.insert(loc=2, column='jahr', value=0)
kerg2_2021['jahr'] = 2021
kerg2_2021['Anzahl'] = kerg2_2021['Anzahl'].map(lambda x: int(x) if not pd.isna(x) else 0)

kerg2_2017 = pd.read_csv(kerg2_2017_csv, sep=';', skiprows=9)
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Gebietsart'] == 'Wahlkreis']
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Stimme'] == 1]
kerg2_2017 = kerg2_2017.loc[kerg2_2017['Gruppenart'] != 'System-Gruppe']
kerg2_2017['Gruppenname'] = kerg2_2017.apply(lambda row: row.Gruppenname + ' '+ str(row.Gebietsnummer) if row.Gruppenart == 'Einzelbewerber/Wählergruppe' else row.Gruppenname, axis=1)
kerg2_2017 = kerg2_2017.filter(items=['Gebietsnummer', 'Gruppenname', 'Anzahl'])
kerg2_2017.insert(loc=2, column='jahr', value=0)
kerg2_2017['jahr'] = 2017
kerg2_2017['Anzahl'] = kerg2_2017['Anzahl'].map(lambda x: int(x) if not pd.isna(x) else 0)

merged = pd.concat([kerg2_2021,kerg2_2017], ignore_index=True)
merged = merged.rename(columns={'Gruppenname':'kurzbezeichnung', 'Gebietsnummer': 'wahlkreis_id', 'Anzahl': 'anzahl'})

parteien = pd.read_csv(parteien_csv, sep=';')
parteien = parteien.rename(columns={'index':'partei_id'})
merged = merged.merge(right=parteien, on=['kurzbezeichnung'], how='inner')
merged = merged.filter(items=['partei_id', 'wahlkreis_id','jahr', 'anzahl'])

merged.to_csv(erststimme_aggr_csv, sep=';', index=False)

In [70]:
# stimmen generator

wahlkreise = list(range(1, 300))
wahlkreise = [1]
years = [2021]

zweitstimmenAggr = pd.read_csv(zweitstimme_aggr_csv, sep=';')
erststimmenAggr = pd.read_csv(erststimme_aggr_csv, sep=';')

stimmenGesamt = pd.DataFrame()

for year in years:
    zweitstimmenAggrJahr = zweitstimmenAggr.loc[zweitstimmenAggr['jahr'] == year]
    erststimmenAggrjahr = erststimmenAggr.loc[erststimmenAggr['jahr'] == year]

    for wahlkreis_id in wahlkreise:
        wahlkreisZweitStimmen = zweitstimmenAggrJahr.loc[zweitstimmenAggrJahr['wahlkreis_id'] == wahlkreis_id]
        wahlkreisErstStimmen = erststimmenAggrjahr.loc[erststimmenAggrjahr['wahlkreis_id'] == wahlkreis_id]

        zweitStimmenWahlkreis = pd.DataFrame()
        erstStimmenWahlkreis = pd.DataFrame()

        for zweitStimmeAggr in wahlkreisZweitStimmen.iterrows():
            zweitStimmeNichtAggr = pd.DataFrame(index=range(0, zweitStimmeAggr[1]['anzahl']))
            zweitStimmeNichtAggr['partei_id'] = zweitStimmeAggr[1]['partei_id']

            zweitStimmenWahlkreis = pd.concat([zweitStimmenWahlkreis, zweitStimmeNichtAggr], ignore_index=True)

        for erstStimmeAggr in wahlkreisErstStimmen.iterrows():
            erstStimmeNichtAggr = pd.DataFrame(index=range(0, erstStimmeAggr[1]['anzahl']))
            erstStimmeNichtAggr['partei_id'] = erstStimmeAggr[1]['partei_id']

            erstStimmenWahlkreis = pd.concat([erstStimmenWahlkreis, erstStimmeNichtAggr], ignore_index=True)
        
        stimmenWahlkreis = erstStimmenWahlkreis.merge(right=zweitStimmenWahlkreis, how='outer', left_index=True, right_index= True, suffixes = ['_erststimme', '_zweitstimme'])
        stimmenWahlkreis['partei_id_zweitstimme'] = stimmenWahlkreis['partei_id_zweitstimme'].map(lambda id: int(id) if not pd.isna(id) else -1)
        stimmenWahlkreis['partei_id_erststimme'] = stimmenWahlkreis['partei_id_erststimme'].map(lambda id: int(id) if not pd.isna(id) else -1)
        stimmenWahlkreis['wahlkreis_id'] = wahlkreis_id
        stimmenWahlkreis['jahr'] = year

        stimmenGesamt = pd.concat([stimmenGesamt, stimmenWahlkreis])

    
stimmenGesamt.to_csv(stimmen_csv, sep=';', index=False)
